# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ing. en Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 06**: Netflix

**Fecha**: 07 de marzo del 2025

**Nombre del Estudiante**: Luis, Benja, Sam

**Profesor**: Pablo Camarillo Ramirez

In [122]:
import findspark
findspark.init()

In [123]:
spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions") \
    .master("spark://b33dcc1265b4:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

In [127]:
from gatubelxs.spark_utils import SparkUtils
netflis_schema = SparkUtils.generate_schema([('show_id','string'),('type','string'),('title','string'),('director','string'),('country','string'),('date_added','string'),('release_year','integer'),('rating','string'),('duration', 'string'),('listed_in','string')])

netflis_df = spark.read \
                .schema(netflis_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/netflix1.csv")

netflis_df.printSchema()

netflis_df.show(5, truncate=False)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)

+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|show_id|type   |title                           |director       |country      |date_added|release_year|rating|duration|listed_in                                                    |
+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|s1     |Movie  |Dick Johnson Is Dead           

In [128]:
null_counts_df = netflis_df.selectExpr(
    *[f"SUM(CASE WHEN {col} IS NULL THEN 1 ELSE 0 END) AS `{col}`" for col in netflis_df.columns]
)

null_counts_df.show()


+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|show_id|type|title|director|country|date_added|release_year|rating|duration|listed_in|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|      0|   0|    0|       1|      1|         1|           2|     1|       2|        2|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+



In [129]:
cleaned_netflis_df = SparkUtils.clean_df(netflis_df)

In [130]:
null_counts_df = cleaned_netflis_df.selectExpr(
    *[f"SUM(CASE WHEN {col} IS NULL THEN 1 ELSE 0 END) AS `{col}`" for col in cleaned_netflis_df.columns]
)

null_counts_df.show()

+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|show_id|type|title|director|country|date_added|release_year|rating|duration|listed_in|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|      0|   0|    0|       0|      0|         0|           0|     0|       0|        0|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+



In [ ]:
filename = 'netflis_o'
SparkUtils.write_df(cleaned_netflis_df,"release_year",filename)

In [121]:
sc.stop()